# Special thanks to 
https://www.chinahadoop.cn/course/1327

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
BATCH_SIZE = 500 
NOISE_DIM = 5

In [3]:
def get_gaussian_dist(mu, sigma): # the real data
    temp = np.random.normal(mu, sigma, size=BATCH_SIZE)[:, np.newaxis]
    return torch.from_numpy(temp).float()

In [4]:
def extract(v):
    return v.detach().storage().tolist()
def stats(d):
    return [np.mean(d), np.std(d)]

In [5]:
# Generator 
G = nn.Sequential(                      
    nn.Linear(NOISE_DIM, 128),            
    nn.ReLU(),
    nn.Linear(128, 1),                  
)

# Discriminator 
D = nn.Sequential(
    nn.Linear(1, 128),     
    nn.ReLU(),
    nn.Linear(128, 1),     
    nn.Sigmoid(),
)

In [ ]:
opt_D = torch.optim.Adam(D.parameters(), lr=0.0001)
opt_G = torch.optim.Adam(G.parameters(), lr=0.0001)

In [ ]:
for step in range(10000):
    # train discrimintor
    d_real_data = get_gaussian_dist(5, 2)    # real data
    noise = torch.randn(BATCH_SIZE, NOISE_DIM)  # random noise
    d_fake_data = G(noise)                      # fake data from G (generated from random ideas)

    prob_real_decision = D(d_real_data)          # D try to increase this prob
    prob_fake_decision = D(d_fake_data.detach()) # D try to reduce this prob
    
    D_loss = - torch.mean(torch.log(prob_real_decision) + torch.log(1. - prob_fake_decision))
    opt_D.zero_grad()
    D_loss.backward()
    opt_D.step()

    # train generator 
    noise = torch.randn(BATCH_SIZE, NOISE_DIM)  # random noise
    g_fake_data = G(noise)                      # fake data from G (generated from random ideas)
    prob_fake_decision = D(g_fake_data)         # G try to increase this prob
    
    G_loss = torch.mean(torch.log(1. - prob_fake_decision))
    opt_G.zero_grad()
    G_loss.backward()
    opt_G.step()

    if step % 500 == 0:  # plotting
        print("Epoch %s: ; Real Dist (%s),  Fake Dist (%s) " %
                  (step, stats(extract(d_real_data)), stats(extract(g_fake_data))))

Epoch 0: ; Real Dist ([5.041547574788332, 2.0625621607974103]),  Fake Dist ([-0.2756656590178609, 0.21883416180330068]) 
Epoch 500: ; Real Dist ([5.049333178859204, 2.0347348151536795]),  Fake Dist ([2.1628397068977354, 0.7166350538622466]) 
Epoch 1000: ; Real Dist ([5.079328663945198, 2.017614950871213]),  Fake Dist ([4.794321310520172, 1.572690531503113]) 
Epoch 1500: ; Real Dist ([4.827821336358785, 2.0167378301132515]),  Fake Dist ([6.4325278205871586, 2.0355453371253014]) 
Epoch 2000: ; Real Dist ([5.026825328994542, 2.1076865328585948]),  Fake Dist ([5.3354990940094, 1.792984728249569]) 
Epoch 2500: ; Real Dist ([4.922828404024243, 1.8385103628352337]),  Fake Dist ([4.604537794828415, 1.4728091747643883]) 
Epoch 3000: ; Real Dist ([5.222733023308217, 2.0222292308826577]),  Fake Dist ([5.374898875236512, 1.8634189113290425]) 
Epoch 3500: ; Real Dist ([4.949746106756851, 2.0439529679785253]),  Fake Dist ([5.088417758941651, 1.978777813689343]) 
Epoch 4000: ; Real Dist ([4.978446275

In [ ]:
print("Plotting the generated distribution...")
values = extract(g_fake_data)
plt.hist(values, bins=50,color="red")
plt.xlabel('Value')
plt.ylabel('Count')
#plt.title('Histogram of Generated Distribution')
plt.grid(True)
plt.show()

In [ ]:
print("Plotting the generated distribution...")
values = extract(d_real_data)
plt.hist(values, bins=50)
plt.xlabel('Value')
plt.ylabel('Count')
plt.title('Histogram of Real Distribution')
plt.grid(True)
plt.show()